# Privilege Escalation - SDCLT UAC Bypass
### Author: Bintang N M
SDCLT is a Microsoft binary that is used in Windows systems (Windows 7 and above) to allow the user to perform backup and restore operations. However it is one of the Microsoft binaries that has been configured to have the autoElevate setting to “true”.


## Use Case
### Objective
Identify UAC bypass by detecting specific registry key access in windows event file.


### Logs Requirements
Microsoft Windows Sysmon

### References
MITRE references: https://attack.mitre.org/techniques/T1548/002/ <br>
sysmon file: https://github.com/Lichtsinnig/EVTX-ATTACK-SAMPLES/blob/master/Privilege%20Escalation/sysmon_1_13_UACBypass_AppPath_Control.evtx and https://github.com/Lichtsinnig/EVTX-ATTACK-SAMPLES/blob/master/Privilege%20Escalation/Sysmon_13_1_UACBypass_SDCLTBypass.evtx <br>
Additional reference: https://pentestlab.blog/2017/06/09/uac-bypass-sdclt/

In [1]:
import evtx
import xml.etree.ElementTree as ET
import json
import re

### SDCLT UAC bypass using App Paths
The backup and restore operation is part of the control panel. This means that when the sdclt.exe process starts the control panel is starting as well. This binary it is designed to run as a high integrity process and it looks for "HKCU\Software\Microsoft\Windows\CurrentVersion\App Paths\control.exe" in the registry which should not be exist by default. However, the attacker can create this registry location and execute the sdclt.exe in order to execute a payload as a high integrity process bypassing the User Account Control.<br><br>
To detect this activity, filter the windows event to show RegistryEvent (event ID: 13) which contain 'control.exe'.

In [3]:
evtx_file = "sysmon_1_13_UACBypass_AppPath_Control.evtx"

parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records_json())

saved_event = []
# print(len(parse_json))

for pj in parse_json:
    if '"EventID": 13' in pj['data'] and 'control.exe' in pj['data']:
        saved_event+=[pj]
        print(pj['data'])

{
  "Event": {
    "#attributes": {
      "xmlns": "http://schemas.microsoft.com/win/2004/08/events/event"
    },
    "EventData": {
      "Details": "C:\\windows\\system32\\cmd.exe",
      "EventType": "SetValue",
      "Image": "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
      "ProcessGuid": "365ABB72-9570-5CD3-0000-00103FC90A00",
      "ProcessId": 1900,
      "RuleName": "",
      "TargetObject": "HKU\\S-1-5-21-3583694148-1414552638-2922671848-1000\\Software\\Microsoft\\Windows\\CurrentVersion\\App Paths\\control.exe\\(Default)",
      "UtcTime": "2019-05-09 03:25:24.552"
    },
    "System": {
      "Channel": "Microsoft-Windows-Sysmon/Operational",
      "Computer": "IEWIN7",
      "Correlation": null,
      "EventID": 13,
      "EventRecordID": 11264,
      "Execution": {
        "#attributes": {
          "ProcessID": 1988,
          "ThreadID": 228
        }
      },
      "Keywords": "0x8000000000000000",
      "Level": 4,
      "Opcode": 0,
      "Provi

### SDCLT UAC bypass Fileless
When the sdclt will executed again with the /kickoffelev parameter it will look for "HKCU\Software\Classes\exefile\shell\runas\command\IsolatedCommand" in the registry which should not be exist by default. However, the attacker can create this registry location and execute the sdclt.exe in order to execute a payload as a high integrity process bypassing the User Account Control.<br><br>
To detect this activity, filter the windows event to show RegistryEvent (event ID: 13) which contain 'command\IsolatedCommand'.

In [5]:
evtx_file = "Sysmon_13_1_UACBypass_SDCLTBypass.evtx"
# evtx_sec = "DE_1102_security_log_cleared.evtx"

parser = evtx.PyEvtxParser(evtx_file)
parse_json = list(parser.records_json())

saved_event = []
# print(len(parse_json))

for pj in parse_json:
    if '"EventID": 13' in pj['data'] and '\\\\command\\\\IsolatedCommand' in pj['data']:
        saved_event+=[pj]
        print(pj['data'])

{
  "Event": {
    "#attributes": {
      "xmlns": "http://schemas.microsoft.com/win/2004/08/events/event"
    },
    "EventData": {
      "Details": "C:\\Windows\\System32\\cmd.exe /c notepad.exe",
      "EventType": "SetValue",
      "Image": "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
      "ProcessGuid": "365ABB72-88DC-5CD3-0000-00100DA51A00",
      "ProcessId": 2704,
      "RuleName": "",
      "TargetObject": "HKU\\S-1-5-21-3583694148-1414552638-2922671848-1000_CLASSES\\exefile\\shell\\runas\\command\\IsolatedCommand",
      "UtcTime": "2019-05-09 02:07:51.100"
    },
    "System": {
      "Channel": "Microsoft-Windows-Sysmon/Operational",
      "Computer": "IEWIN7",
      "Correlation": null,
      "EventID": 13,
      "EventRecordID": 11122,
      "Execution": {
        "#attributes": {
          "ProcessID": 1980,
          "ThreadID": 1904
        }
      },
      "Keywords": "0x8000000000000000",
      "Level": 4,
      "Opcode": 0,
      "Provider": {
